# Fine-tune and Evaluate LoRA Model

This notebook orchestrates the complete workflow:
1. Hugging Face authentication
2. Model training using config.yaml
3. Model evaluation
4. Upload trained model to Hugging Face Hub


In [ ]:
# Import required libraries
import os
import wandb
from dotenv import load_dotenv

# Import utility functions
from utils.config_utils import load_config
from utils.data_utils import load_and_prepare_dataset
from utils.hf_utils import authenticate_huggingface
from utils.model_utils import setup_model_and_tokenizer
from train_qlora import train_model
from evaludate_qlora import evaluate_peft_model
from paths import OUTPUTS_DIR

# Load environment variables
load_dotenv()
os.environ["TOKENIZERS_PARALLELISM"] = "false"

print("✅ All imports successful!")


## Step 1: Authenticate with Hugging Face

Authenticate to access gated models and upload your trained model.


In [ ]:
# Authenticate with Hugging Face
# This will check for HUGGINGFACE_TOKEN or HF_TOKEN environment variables
# Or prompt for interactive login
authenticate_huggingface()


## Step 2: Load Configuration

Load model and training configuration from config.yaml


In [ ]:
# Load configuration from config.yaml
cfg = load_config()

print("📋 Configuration loaded:")
print(f"  Base Model: {cfg['base_model']}")
print(f"  Dataset: {cfg['dataset']['name']}")
print(f"  Learning Rate: {cfg['learning_rate']}")
print(f"  Batch Size: {cfg['batch_size']}")
print(f"  Epochs: {cfg['num_epochs']}")
print(f"  LoRA r: {cfg['lora_r']}")
print(f"  LoRA alpha: {cfg['lora_alpha']}")


## Step 3: Load Dataset

Load and prepare the training and validation datasets.


In [ ]:
# Load dataset splits
train_data, val_data, test_data = load_and_prepare_dataset(cfg)

print(f"✅ Dataset loaded:")
print(f"  Training samples: {len(train_data)}")
print(f"  Validation samples: {len(val_data)}")
print(f"  Test samples: {len(test_data)}")


## Step 4: Setup Model and Tokenizer

Initialize the base model with quantization and LoRA configuration.


In [ ]:
# Setup model with 4-bit quantization and LoRA
model, tokenizer = setup_model_and_tokenizer(
    cfg, 
    use_4bit=True, 
    use_lora=True, 
    padding_side="right"
)

print("✅ Model and tokenizer ready for training!")


## Step 5: Initialize Weights & Biases (Optional)

Initialize W&B for experiment tracking.


In [ ]:
# Initialize W&B for experiment tracking
wandb.init(
    project=cfg.get("wandb_project", "samsum"),
    name=cfg.get("wandb_run_name", "lora-finetuning-default-hps"),
    config={
        "model": cfg["base_model"],
        "learning_rate": cfg.get("learning_rate", 2e-4),
        "epochs": cfg.get("num_epochs", 1),
        "lora_r": cfg.get("lora_r", 8),
        "lora_alpha": cfg.get("lora_alpha", 16),
    },
)

print("✅ W&B initialized!")


## Step 6: Train the Model

Fine-tune the model using LoRA on the training dataset.


In [ ]:
# Train the model
train_model(
    cfg,
    model,
    tokenizer,
    train_data,
    val_data,
    save_dir=cfg.get("save_dir", None),
)

# Finish W&B run
wandb.finish()

print("\n✅ Training complete!")


## Step 7: Evaluate the Trained Model

Evaluate the fine-tuned model on the validation set using ROUGE metrics.


In [ ]:
# Evaluate the trained model
# The adapter_dir will default to OUTPUTS_DIR/lora_samsum/lora_adapters
scores, preds = evaluate_peft_model(cfg)

print("\n📊 Final Evaluation Results:")
print(f"  ROUGE-1: {scores['rouge1']:.2%}")
print(f"  ROUGE-2: {scores['rouge2']:.2%}")
print(f"  ROUGE-L: {scores['rougeL']:.2%}")

# Show a sample prediction
print("\n📝 Sample Prediction:")
print(f"  Prediction: {preds[0]}")


## Step 8: Upload Model to Hugging Face Hub

Upload the trained LoRA adapters to Hugging Face Hub.


In [ ]:
from huggingface_hub import HfApi, create_repo

# Set your Hugging Face username and model name
# TODO: Update these with your Hugging Face username and desired model name
HF_USERNAME = "your-username"  # Replace with your Hugging Face username
MODEL_NAME = "llama-3.2-1b-samsum-lora"  # Replace with your desired model name
HF_REPO_ID = f"{HF_USERNAME}/{MODEL_NAME}"

print(f"📤 Preparing to upload model to: {HF_REPO_ID}")


In [ ]:
# Get the adapter directory path
adapter_dir = os.path.join(OUTPUTS_DIR, "lora_samsum", "lora_adapters")

if not os.path.exists(adapter_dir):
    raise FileNotFoundError(f"❌ Adapter directory not found: {adapter_dir}")

print(f"📂 Adapter directory: {adapter_dir}")

# Create repository on Hugging Face Hub (if it doesn't exist)
api = HfApi()
try:
    create_repo(
        repo_id=HF_REPO_ID,
        repo_type="model",
        private=False,  # Set to True if you want a private repo
        exist_ok=True,
    )
    print(f"✅ Repository created/verified: {HF_REPO_ID}")
except Exception as e:
    print(f"⚠️  Repository creation note: {e}")


In [ ]:
# Upload the adapter files to Hugging Face Hub
print(f"📤 Uploading adapter files to {HF_REPO_ID}...")

api.upload_folder(
    folder_path=adapter_dir,
    repo_id=HF_REPO_ID,
    repo_type="model",
    commit_message=f"Upload LoRA adapters fine-tuned on SAMSum dataset\n\nROUGE-1: {scores['rouge1']:.2%}\nROUGE-2: {scores['rouge2']:.2%}\nROUGE-L: {scores['rougeL']:.2%}",
)

print(f"✅ Model successfully uploaded to: https://huggingface.co/{HF_REPO_ID}")


## Summary

✅ Training and evaluation complete!

- **Model**: Fine-tuned LoRA adapters saved locally
- **Evaluation**: ROUGE scores computed and saved
- **Hugging Face**: Model uploaded to Hub

You can now use your fine-tuned model from Hugging Face Hub:
```python
from peft import PeftModel
from transformers import AutoModelForCausalLM, AutoTokenizer

base_model = AutoModelForCausalLM.from_pretrained(cfg["base_model"])
model = PeftModel.from_pretrained(base_model, f"{HF_USERNAME}/{MODEL_NAME}")
```
